# Regression Exercise 

California Housing Data

This data set contains information about all the block groups in California from the 1990 Census. In this sample a block group on average includes 1425.5 individuals living in a geographically compact area. 

The task is to aproximate the median house value of each block from the values of the rest of the variables. 

 It has been obtained from the LIACC repository. The original page where the data set can be found is: http://www.liaad.up.pt/~ltorgo/Regression/DataSets.html.
 

The Features:
 
* housingMedianAge: continuous. 
* totalRooms: continuous. 
* totalBedrooms: continuous. 
* population: continuous. 
* households: continuous. 
* medianIncome: continuous. 
* medianHouseValue: continuous. 

## The Data

** Import the cal_housing_clean.csv file with pandas. Separate it into a training (70%) and testing set(30%).**

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('./cal_housing_clean.csv')

In [3]:
df.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [4]:
df.describe()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,28.639486,2635.763081,537.898014,1425.476744,499.539680,3.870671,206855.816909
std,12.585558,2181.615252,421.247906,1132.462122,382.329753,1.899822,115395.615874
min,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,18.000000,1447.750000,295.000000,787.000000,280.000000,2.563400,119600.000000
50%,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [5]:
from sklearn.cross_validation import train_test_split

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [6]:
X_data=df.drop(['medianHouseValue'],axis=1)
y_data=df['medianHouseValue']

In [7]:
X_train, X_test, y_train, y_test=train_test_split(X_data,y_data,test_size=0.3, random_state=102)

In [8]:
from sklearn.preprocessing import MinMaxScaler

In [9]:
scaler=MinMaxScaler()

In [10]:
scaler.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

### Scale the Feature Data

** Use sklearn preprocessing to create a MinMaxScaler for the feature data. Fit this scaler only to the training data. Then use it to transform X_test and X_train. Then use the scaled X_test and X_train along with pd.Dataframe to re-create two dataframes of scaled data.**

In [11]:
X_train = pd.DataFrame(data=scaler.transform(X_train),columns = X_train.columns,index=X_train.index)

In [12]:
X_test = pd.DataFrame(data=scaler.transform(X_test),columns = X_test.columns,index=X_test.index)

In [13]:
import tensorflow as tf

### Create Feature Columns

** Create the necessary tf.feature_column objects for the estimator. They should all be trated as continuous numeric_columns. **

In [14]:
df.columns

Index([u'housingMedianAge', u'totalRooms', u'totalBedrooms', u'population',
       u'households', u'medianIncome', u'medianHouseValue'],
      dtype='object')

In [15]:
medianAge=tf.feature_column.numeric_column('housingMedianAge')
rooms = tf.feature_column.numeric_column('totalRooms')
bedrooms = tf.feature_column.numeric_column('totalBedrooms')
pop = tf.feature_column.numeric_column('population')
households = tf.feature_column.numeric_column('households')
income = tf.feature_column.numeric_column('medianIncome')

In [16]:
feat_cols = [ medianAge,rooms,bedrooms,pop,households,income]

** Create the input function for the estimator object. (play around with batch_size and num_epochs)**

In [17]:
train_input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=10,num_epochs=1000,shuffle=True)

** Create the estimator model. Use a DNNRegressor. Play around with the hidden units! **

In [18]:
model=tf.estimator.DNNRegressor(hidden_units=[6,7,6,7],feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6a66c70c50>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/tmp/tmpnRfzhL', '_save_summary_steps': 100}


##### ** Train the model for ~1,000 steps. (Later come back to this and train it for more and check for improvement) **

In [19]:
model.train(input_fn=train_input_func,steps=200000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpnRfzhL/model.ckpt.
INFO:tensorflow:loss = 5.36557e+11, step = 1
INFO:tensorflow:global_step/sec: 294.685
INFO:tensorflow:loss = 3.70281e+11, step = 101 (0.341 sec)
INFO:tensorflow:global_step/sec: 324.607
INFO:tensorflow:loss = 3.54539e+11, step = 201 (0.308 sec)
INFO:tensorflow:global_step/sec: 321.624
INFO:tensorflow:loss = 1.33386e+11, step = 301 (0.311 sec)
INFO:tensorflow:global_step/sec: 346.828
INFO:tensorflow:loss = 2.48851e+11, step = 401 (0.289 sec)
INFO:tensorflow:global_step/sec: 321.715
INFO:tensorflow:loss = 6.39441e+10, step = 501 (0.311 sec)
INFO:tensorflow:global_step/sec: 313.146
INFO:tensorflow:loss = 1.49004e+11, step = 601 (0.317 sec)
INFO:tensorflow:global_step/sec: 299.635
INFO:tensorflow:loss = 4.52828e+10, step = 701 (0.336 sec)
INFO:tensorflow:global_step/sec: 304.556
INFO:tensorflow:loss = 7.71226e+10, step = 801 (0.327 sec)
INFO:tensorflow:global_step/sec: 323.5

INFO:tensorflow:global_step/sec: 343.884
INFO:tensorflow:loss = 5.5223e+10, step = 8101 (0.292 sec)
INFO:tensorflow:global_step/sec: 326.165
INFO:tensorflow:loss = 1.152e+11, step = 8201 (0.306 sec)
INFO:tensorflow:global_step/sec: 326.478
INFO:tensorflow:loss = 1.21765e+11, step = 8301 (0.306 sec)
INFO:tensorflow:global_step/sec: 325.724
INFO:tensorflow:loss = 9.32027e+10, step = 8401 (0.308 sec)
INFO:tensorflow:global_step/sec: 321.251
INFO:tensorflow:loss = 4.90571e+10, step = 8501 (0.311 sec)
INFO:tensorflow:global_step/sec: 329.14
INFO:tensorflow:loss = 7.17462e+10, step = 8601 (0.304 sec)
INFO:tensorflow:global_step/sec: 321.492
INFO:tensorflow:loss = 4.40882e+10, step = 8701 (0.310 sec)
INFO:tensorflow:global_step/sec: 329.585
INFO:tensorflow:loss = 1.20874e+11, step = 8801 (0.304 sec)
INFO:tensorflow:global_step/sec: 317.903
INFO:tensorflow:loss = 1.23838e+11, step = 8901 (0.315 sec)
INFO:tensorflow:global_step/sec: 325.881
INFO:tensorflow:loss = 9.05357e+10, step = 9001 (0.307

INFO:tensorflow:global_step/sec: 318.187
INFO:tensorflow:loss = 5.54543e+10, step = 16201 (0.315 sec)
INFO:tensorflow:global_step/sec: 322.057
INFO:tensorflow:loss = 7.28655e+10, step = 16301 (0.310 sec)
INFO:tensorflow:global_step/sec: 321.525
INFO:tensorflow:loss = 1.10781e+11, step = 16401 (0.312 sec)
INFO:tensorflow:global_step/sec: 309.105
INFO:tensorflow:loss = 6.07457e+10, step = 16501 (0.323 sec)
INFO:tensorflow:global_step/sec: 319.311
INFO:tensorflow:loss = 1.47571e+11, step = 16601 (0.313 sec)
INFO:tensorflow:global_step/sec: 331.633
INFO:tensorflow:loss = 9.15874e+10, step = 16701 (0.302 sec)
INFO:tensorflow:global_step/sec: 322.609
INFO:tensorflow:loss = 6.26457e+10, step = 16801 (0.311 sec)
INFO:tensorflow:global_step/sec: 306.324
INFO:tensorflow:loss = 5.11788e+10, step = 16901 (0.326 sec)
INFO:tensorflow:global_step/sec: 326.158
INFO:tensorflow:loss = 5.66779e+10, step = 17001 (0.307 sec)
INFO:tensorflow:global_step/sec: 329.411
INFO:tensorflow:loss = 6.32198e+10, step 

INFO:tensorflow:global_step/sec: 321.328
INFO:tensorflow:loss = 3.01746e+10, step = 24301 (0.312 sec)
INFO:tensorflow:global_step/sec: 342.386
INFO:tensorflow:loss = 9.54479e+10, step = 24401 (0.291 sec)
INFO:tensorflow:global_step/sec: 317.093
INFO:tensorflow:loss = 1.37172e+11, step = 24501 (0.317 sec)
INFO:tensorflow:global_step/sec: 340.775
INFO:tensorflow:loss = 1.29521e+11, step = 24601 (0.292 sec)
INFO:tensorflow:global_step/sec: 323.626
INFO:tensorflow:loss = 3.26251e+10, step = 24701 (0.309 sec)
INFO:tensorflow:global_step/sec: 335.44
INFO:tensorflow:loss = 4.66743e+10, step = 24801 (0.298 sec)
INFO:tensorflow:global_step/sec: 319.561
INFO:tensorflow:loss = 5.93317e+10, step = 24901 (0.313 sec)
INFO:tensorflow:global_step/sec: 332.409
INFO:tensorflow:loss = 4.01191e+10, step = 25001 (0.301 sec)
INFO:tensorflow:global_step/sec: 317.277
INFO:tensorflow:loss = 3.87977e+10, step = 25101 (0.315 sec)
INFO:tensorflow:global_step/sec: 329.063
INFO:tensorflow:loss = 6.79167e+10, step =

INFO:tensorflow:global_step/sec: 329.12
INFO:tensorflow:loss = 7.76015e+10, step = 32401 (0.304 sec)
INFO:tensorflow:global_step/sec: 343.978
INFO:tensorflow:loss = 7.83045e+10, step = 32501 (0.290 sec)
INFO:tensorflow:global_step/sec: 351.665
INFO:tensorflow:loss = 3.56012e+10, step = 32601 (0.284 sec)
INFO:tensorflow:global_step/sec: 356.558
INFO:tensorflow:loss = 1.17626e+11, step = 32701 (0.281 sec)
INFO:tensorflow:global_step/sec: 333.497
INFO:tensorflow:loss = 1.45198e+11, step = 32801 (0.299 sec)
INFO:tensorflow:global_step/sec: 317.285
INFO:tensorflow:loss = 5.81507e+10, step = 32901 (0.316 sec)
INFO:tensorflow:global_step/sec: 342.13
INFO:tensorflow:loss = 5.87104e+10, step = 33001 (0.292 sec)
INFO:tensorflow:global_step/sec: 323.741
INFO:tensorflow:loss = 4.83009e+10, step = 33101 (0.309 sec)
INFO:tensorflow:global_step/sec: 334.737
INFO:tensorflow:loss = 7.69659e+10, step = 33201 (0.299 sec)
INFO:tensorflow:global_step/sec: 321.23
INFO:tensorflow:loss = 1.03884e+11, step = 3

...

INFO:tensorflow:global_step/sec: 339.177
INFO:tensorflow:loss = 3.62545e+10, step = 161201 (0.294 sec)
INFO:tensorflow:global_step/sec: 334.133
INFO:tensorflow:loss = 5.20605e+10, step = 161301 (0.300 sec)
INFO:tensorflow:global_step/sec: 323.849
INFO:tensorflow:loss = 3.99918e+10, step = 161401 (0.309 sec)
INFO:tensorflow:global_step/sec: 326.941
INFO:tensorflow:loss = 2.97603e+10, step = 161501 (0.306 sec)
INFO:tensorflow:global_step/sec: 319.256
INFO:tensorflow:loss = 7.90641e+10, step = 161601 (0.313 sec)
INFO:tensorflow:global_step/sec: 327.292
INFO:tensorflow:loss = 2.96144e+10, step = 161701 (0.306 sec)
INFO:tensorflow:global_step/sec: 321.835
INFO:tensorflow:loss = 8.57622e+10, step = 161801 (0.310 sec)
INFO:tensorflow:global_step/sec: 303.238
INFO:tensorflow:loss = 1.0944e+11, step = 161901 (0.330 sec)
INFO:tensorflow:global_step/sec: 321.449
INFO:tensorflow:loss = 5.57748e+10, step = 162001 (0.311 sec)
INFO:tensorflow:global_step/sec: 335.752
INFO:tensorflow:loss = 6.10661e+1

INFO:tensorflow:global_step/sec: 328.197
INFO:tensorflow:loss = 4.31956e+10, step = 169201 (0.304 sec)
INFO:tensorflow:global_step/sec: 326.604
INFO:tensorflow:loss = 4.60563e+10, step = 169301 (0.306 sec)
INFO:tensorflow:global_step/sec: 321.79
INFO:tensorflow:loss = 3.16769e+10, step = 169401 (0.311 sec)
INFO:tensorflow:global_step/sec: 325.576
INFO:tensorflow:loss = 1.38986e+11, step = 169501 (0.307 sec)
INFO:tensorflow:global_step/sec: 333.665
INFO:tensorflow:loss = 7.57324e+10, step = 169601 (0.299 sec)
INFO:tensorflow:global_step/sec: 326.811
INFO:tensorflow:loss = 9.27002e+10, step = 169701 (0.306 sec)
INFO:tensorflow:global_step/sec: 320.359
INFO:tensorflow:loss = 7.79083e+10, step = 169801 (0.313 sec)
INFO:tensorflow:global_step/sec: 305.358
INFO:tensorflow:loss = 2.16858e+10, step = 169901 (0.327 sec)
INFO:tensorflow:global_step/sec: 315.718
INFO:tensorflow:loss = 4.37896e+10, step = 170001 (0.317 sec)
INFO:tensorflow:global_step/sec: 335.633
INFO:tensorflow:loss = 8.95778e+1

INFO:tensorflow:global_step/sec: 336.976
INFO:tensorflow:loss = 1.38941e+10, step = 177201 (0.297 sec)
INFO:tensorflow:global_step/sec: 320.408
INFO:tensorflow:loss = 3.52105e+10, step = 177301 (0.312 sec)
INFO:tensorflow:global_step/sec: 332.085
INFO:tensorflow:loss = 4.64764e+10, step = 177401 (0.301 sec)
INFO:tensorflow:global_step/sec: 311.664
INFO:tensorflow:loss = 3.27459e+10, step = 177501 (0.321 sec)
INFO:tensorflow:global_step/sec: 318.446
INFO:tensorflow:loss = 5.88585e+10, step = 177601 (0.314 sec)
INFO:tensorflow:global_step/sec: 325.187
INFO:tensorflow:loss = 9.30439e+10, step = 177701 (0.307 sec)
INFO:tensorflow:global_step/sec: 327.128
INFO:tensorflow:loss = 1.88109e+10, step = 177801 (0.306 sec)
INFO:tensorflow:global_step/sec: 331.937
INFO:tensorflow:loss = 7.15963e+10, step = 177901 (0.301 sec)
INFO:tensorflow:global_step/sec: 324.583
INFO:tensorflow:loss = 1.91587e+10, step = 178001 (0.308 sec)
INFO:tensorflow:global_step/sec: 328.545
INFO:tensorflow:loss = 2.0897e+1

INFO:tensorflow:global_step/sec: 320.698
INFO:tensorflow:loss = 4.95639e+10, step = 185201 (0.311 sec)
INFO:tensorflow:global_step/sec: 328.148
INFO:tensorflow:loss = 1.81157e+11, step = 185301 (0.305 sec)
INFO:tensorflow:global_step/sec: 334.996
INFO:tensorflow:loss = 1.26244e+11, step = 185401 (0.298 sec)
INFO:tensorflow:global_step/sec: 331.344
INFO:tensorflow:loss = 1.45687e+11, step = 185501 (0.302 sec)
INFO:tensorflow:global_step/sec: 318.674
INFO:tensorflow:loss = 3.21289e+10, step = 185601 (0.314 sec)
INFO:tensorflow:global_step/sec: 310.505
INFO:tensorflow:loss = 7.14725e+10, step = 185701 (0.323 sec)
INFO:tensorflow:global_step/sec: 346.128
INFO:tensorflow:loss = 2.97379e+10, step = 185801 (0.289 sec)
INFO:tensorflow:global_step/sec: 326.461
INFO:tensorflow:loss = 2.28218e+10, step = 185901 (0.306 sec)
INFO:tensorflow:global_step/sec: 326.654
INFO:tensorflow:loss = 4.26283e+10, step = 186001 (0.305 sec)
INFO:tensorflow:global_step/sec: 319.774
INFO:tensorflow:loss = 1.66042e+

INFO:tensorflow:global_step/sec: 338.051
INFO:tensorflow:loss = 2.13219e+10, step = 193201 (0.297 sec)
INFO:tensorflow:global_step/sec: 317.373
INFO:tensorflow:loss = 4.00894e+10, step = 193301 (0.317 sec)
INFO:tensorflow:global_step/sec: 312.254
INFO:tensorflow:loss = 3.06604e+10, step = 193401 (0.317 sec)
INFO:tensorflow:global_step/sec: 327.286
INFO:tensorflow:loss = 4.1082e+10, step = 193501 (0.305 sec)
INFO:tensorflow:global_step/sec: 321.492
INFO:tensorflow:loss = 6.18908e+10, step = 193601 (0.312 sec)
INFO:tensorflow:global_step/sec: 312.635
INFO:tensorflow:loss = 2.79355e+10, step = 193701 (0.320 sec)
INFO:tensorflow:global_step/sec: 309.638
INFO:tensorflow:loss = 8.46222e+10, step = 193801 (0.323 sec)
INFO:tensorflow:global_step/sec: 327.593
INFO:tensorflow:loss = 4.87627e+10, step = 193901 (0.306 sec)
INFO:tensorflow:global_step/sec: 330.62
INFO:tensorflow:loss = 9.94908e+10, step = 194001 (0.302 sec)
INFO:tensorflow:global_step/sec: 323.945
INFO:tensorflow:loss = 1.48586e+11

** Create a prediction input function and then use the .predict method off your estimator model to create a list or predictions on your test data. **

In [20]:
predict_input_func = tf.estimator.inputs.pandas_input_fn(
      x=X_test,
      batch_size=10,
      num_epochs=1,
      shuffle=False)

In [21]:
pred_gen = model.predict(predict_input_func)

In [22]:
predictions = list(pred_gen)

INFO:tensorflow:Restoring parameters from /tmp/tmpnRfzhL/model.ckpt-200000


** Calculate the RMSE. You should be able to get around 100,000 RMSE (remember that this is in the same units as the label.) Do this manually or use [sklearn.metrics](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) **

In [23]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['predictions'])

In [24]:
from sklearn.metrics import mean_squared_error

In [25]:
mean_squared_error(y_test,final_preds)**0.5

78387.950104955482

# Great Job!